<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html week15.do.txt --no_mako --no_abort -->
<!-- dom:TITLE: Deep learning and many-body problems -->

# Deep learning and many-body problems
**Morten Hjorth-Jensen  Email morten.hjorth-jensen@fys.uio.no**, Department of Physics and Center fo Computing in Science Education, University of Oslo, Oslo, Norway

Date: **May 9**

## Overview of week of May 5-9
* Neural networks and project 2

a. Discussions of neural network and solution of project 2